## Select 500 HQ MAGs and write paths to directory ##
- want 100 without proviruses

In [1]:
import os
from csv import DictReader
from os.path import join

quality_report = "/mnt/hpccs01/work/microbiome/abisko/groopm/60_Emerge_MAGs_v4/Field/Aroney_v3/checkm2/quality_report.tsv"
genome_dir = "/mnt/hpccs01/work/microbiome/abisko/groopm/60_Emerge_MAGs_v4/Field/Aroney_v3/genomes"

In [2]:
i = 0
with open("hq_mag_filepaths.txt", "w") as w:
    with open(quality_report, "r") as f:
        reader = DictReader(f, delimiter="\t")
        for row in reader:
            if row["Completeness"] >= "95" and row["Contamination"] <= "5":
                genome_name = row["Name"]
                genome_path = join(genome_dir, genome_name + ".fna")
                if os.path.exists(genome_path):
                    w.write(genome_name + "\t" + genome_path + "\n")
                    i += 1
                else:
                    print(f"Warning: {genome_path} does not exist.")
i

9951

In [3]:
# i = 0
# j = 0
# genomes = []
# for folder in os.listdir("genomad"):
#     if not os.path.isdir(join("genomad", folder)):
#         continue
#     provirus_file = join("genomad", folder, f"{folder}_find_proviruses", f"{folder}_provirus.tsv")
#     if not os.path.exists(provirus_file):
#         # print(f"Warning: {provirus_file} does not exist.")
#         continue
#     i += 1
#     with open(provirus_file, "r") as f:
#         next(f)
#         if len(f.readlines()) > 0:
#             j += 1
#         else:
#             genomes.append(folder)
# print(f"{i} folders processed, with {j} containing proviruses.")


In [3]:
i = 0
j = 0
genomes = []
for folder in os.listdir("genomad"):
    if not os.path.isdir(join("genomad", folder)):
        continue
    genome_file = join("genomad", folder, f"{folder}_summary", f"{folder}_virus_summary.tsv")
    if not os.path.exists(genome_file):
        # print(f"Warning: {genome_file} does not exist.")
        continue
    i += 1
    with open(genome_file, "r") as f:
        next(f)
        if len(f.readlines()) > 0:
            j += 1
        else:
            genomes.append(folder)
print(f"{i} folders processed, with {j} containing proviruses.")
        

9951 folders processed, with 4927 containing proviruses.


In [4]:
import pandas as pd
arc_tax = "/mnt/hpccs01/work/microbiome/abisko/groopm/60_Emerge_MAGs_v4/Field/Aroney_v3/gtdbtk/classify/gtdbtk.ar53.summary.tsv"
bac_tax = "/mnt/hpccs01/work/microbiome/abisko/groopm/60_Emerge_MAGs_v4/Field/Aroney_v3/gtdbtk/classify/gtdbtk.bac120.summary.tsv"

headers = ["user_genome", "classification", "genome_size"]
# read quality report to get genome sizes
with open(quality_report, "r") as f:
    reader = DictReader(f, delimiter="\t")
    genome_sizes = {row["Name"]: row["Genome_Size"] for row in reader}

# read arc taxonomy
arc_df = pd.read_csv(arc_tax, sep="\t", usecols=["user_genome", "classification"])
# read bac taxonomy
bac_df = pd.read_csv(bac_tax, sep="\t", usecols=["user_genome", "classification"])
# merge both dataframes
merged_df = pd.concat([arc_df, bac_df], ignore_index=True)
# filter for genomes without proviruses
filtered_df = merged_df[merged_df["user_genome"].isin(genomes)]
# add genome sizes
filtered_df["genome_size"] = filtered_df["user_genome"].map(genome_sizes)
# save to file
filtered_df.to_csv("genomes_without_proviruses_taxonomy.tsv", sep="\t", index=False, columns=headers)

/tmp/ipykernel_2520377/930150255.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["genome_size"] = filtered_df["user_genome"].map(genome_sizes)


In [6]:
# select novel aka species level classification ends with just "s__"
novel_genomes = filtered_df[filtered_df["classification"].str.endswith("g__;s__")]
# novel_genomes = filtered_df[filtered_df["classification"].str.endswith("s__")]
novel_genomes

,user_genome,classification,genome_size
346,binchicken_emerge_co106.403,d__Archaea;p__Thermoproteota;c__Bathyarchaeia;...,1470543
658,binchicken_emerge_co185.459,d__Archaea;p__Thermoplasmatota;c__Thermoplasma...,2483117
742,binchicken_emerge_co217.349,d__Archaea;p__Thermoplasmatota;c__Thermoplasma...,2707623
1355,binchicken_emerge_co82.78,d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...,976003
1843,binchicken_fen_co180.296,d__Archaea;p__Thermoplasmatota;c__Thermoplasma...,1984192
...,...,...,...
57941,binchicken_palsa_co192.599,d__Bacteria;p__Eremiobacterota;c__Eremiobacter...,2986257
58180,binchicken_palsa_co198.373,d__Bacteria;p__Myxococcota;c__Myxococcia;o__My...,5126854
58267,binchicken_palsa_co199.190,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,3554215
58272,binchicken_palsa_co199.21,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,4138486


In [8]:
with open("novel_genomes_without_proviruses.txt", 'w') as w:
    # for genome in genomes:
    #     w.write(genome + "\t" + join(genome_dir, genome + ".fna") + "\n")
    for _, row in novel_genomes.iterrows():
        genome_name = row["user_genome"]
        genome_path = join(genome_dir, genome_name + ".fna")
        if os.path.exists(genome_path):
            w.write(genome_name + "\t" + genome_path + "\n")
        else:
            print(f"Warning: {genome_path} does not exist.")

with open("novel_genomes_without_proviruses_taxonomy.tsv", 'w') as w:
    novel_genomes.to_csv(w, sep="\t", index=False, columns=headers)